In [1]:
!pip install accelerate
!pip install bitsandbytes
!pip install transformers==4.32.0 accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed
!pip install sentence_transformers
!pip install faiss-cpu

  Obtaining dependency information for bitsandbytes from https://files.pythonhosted.org/packages/c2/49/557f8f4aa9cfc1e9d7875fd850a44a6d3d881a42c483bc8cf56a6b597dfe/bitsandbytes-0.41.2.post2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 14.6 MB/s eta 0:00:00
  Obtaining dependency information for transformers==4.32.0 from https://files.pythonhosted.org/packages/ae/95/283a1c004430bd2a9425d6937fc545dd49a4e4592feb76be0299a14e2378/transformers-4.32.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 7.0 MB/s eta 0:00:00
  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/f4/2e/0adf6e264b996e263b1c57cad6560ffd5492a69beb9fd779ed0463d486bc/tiktoken-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for einops from https://files.pythonhosted.org/packages/29/0b/2d1c0ebfd092e25935b86509a9a817159212d82aa43d7fb07eca4eeff2c2/einop

In [2]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 53.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import faiss
import json 
import os
import time 
import torch
from tqdm import tqdm
from pyvi.ViTokenizer import tokenize

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from sentence_transformers import SentenceTransformer
model_embedding = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')
PATH_TRAIN_CSV = "/kaggle/input/zalo-ai-2023-elementaty-maths-solving/zalo_ai_2023_elementary_maths_solving/math_train.json"
PATH_TEST_CSV = "/kaggle/input/zalo-ai-2023-elementaty-maths-solving/zalo_ai_2023_elementary_maths_solving/math_test.json"


In [5]:
def build_faiss_index(model_embedding, path_csv):
    search_faiss = faiss.IndexFlatIP(768)
    data = json.load(open(path_csv, 'r'))
    for item in tqdm(data["data"]):
        sentences = [tokenize(item['question'])]
        embedding =  model_embedding.encode(sentences, show_progress_bar = False)
        embedding = np.array(embedding).astype('float32')
        search_faiss.add(embedding)
    return search_faiss

In [6]:
search_faiss = build_faiss_index(model_embedding, PATH_TRAIN_CSV)

100%|██████████| 1200/1200 [00:20<00:00, 57.57it/s]


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import os

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-14B", trust_remote_code=True)

# use bf16
max_memory_mapping = {0: "16GB"}
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-14B",
                                             device_map="auto",
                                             trust_remote_code=True,
                                             load_in_4bit=True,
#                                              load_in_8bit=False,
                                             # offload_state_dict=True,
                                             # low_cpu_mem_usage=True,
                                             # bf16=True,
                                             max_memory=max_memory_mapping).eval()


A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-14B:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-14B:
- configuration_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-14B:
- qwen_generation_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-14B:
- cpp_kernels.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-14B:
- modeling_qwen.py
- qwen_generation_utils.py
- cpp_kernels.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [8]:
# prompt = '''
# You are a virtual assistant capable of answering math questions honestly and accurately, without fabricating additional content.
# Based on the following multiple choice questions, let's think step by step, choose the correct answer and explain your answer.

# Câu hỏi 1:
# Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính?
# A. 80,000 : 6
# B. 80,000 x 6
# C. 80,000 : (6 x 100)
# D. (80,000 x 6) : 100

# Correct answer: D. (80,000 x 6) : 100
# Explanation: Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồng . Để tìm số tiền lỗ ta có thể lấy 80 000 chia cho 100 rồi nhân với 6 (tức là 80 000 : 100 × 6) hoặc lấy 80000 nhân với 6 rồi chia cho 100 (tức là 80 000 × 6 : 100).

# Câu hỏi 2:
# 8 dm2 24 cm2 = ……… dm2. Số thích hợp điền vào chỗ chấm là?
# A. 824
# B. 82,4
# C. 8,24
# D. 0,824

# Correct answer: C. 8,24
# Explanation: Ta có 24 cm2 = 0,24 dm2\n Vậy 8 dm2 24 cm2 = 8,24 dm2.


# Câu hỏi 3:
# 10% của 11,5m2 là?
# A. 10,15dm2
# B. 1,5m2
# C. 15,5m2
# D. 1,15m2
# Correct answer: D. 1,15m2
# Explanation: 10% của 11,5m2 là: 11,5 ${\\times}$ 10 : 100 = 1,15 (m2).

# Câu hỏi 4:
# Một tam giác có đáy bằng 15 cm và gấp 2,5 lần chiều cao tương ứng. Diện tích tam giác đó là?
# A. 90 cm2
# B. 37,5 cm2
# C. 45 cm2
# D. 18,75 cm2

# Correct answer: C. 45 cm2
# Explanation: Chiều cao của tam giác đó là: 15 : 2,5 = 6 (cm). Diện tích tam giác đó là: 15 ${\\times}$ 6 : 2 = 45 (cm2). Đáp số: 45 cm2.

# Câu hỏi 5:
# Số 13 076 làm tròn đến chữ số hàng chục nghìn ta được số?
# A. 10 000
# B. 13 000
# C. 13 050
# D. 20 000

# Correct answer: A. 10 000
# Explanation: Số 13 076 gần với số 10 000 hơn số 20 000. Vậy khi làm tròn số 13 076 đến hàng chục nghìn ta được số 10 000.

# Câu hỏi 6:

# '''

In [9]:
prefix_prompt = '''
You are a virtual assistant capable of answering math questions honestly and accurately, without fabricating additional content.
Based on the following multiple choice questions, let's think step by step, come up with a solution and choose the correct answer.

Câu hỏi:
Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính?
A. 80,000 : 6
B. 80,000 x 6
C. 80,000 : (6 x 100)
D. (80,000 x 6) : 100
Solution: Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồng . Để tìm số tiền lỗ ta có thể lấy 80 000 chia cho 100 rồi nhân với 6 (tức là 80 000 : 100 × 6) hoặc lấy 80000 nhân với 6 rồi chia cho 100 (tức là 80 000 × 6 : 100).
Correct answer: D. (80,000 x 6) : 100

Câu hỏi:
8 dm2 24 cm2 = ……… dm2. Số thích hợp điền vào chỗ chấm là?
A. 824
B. 82,4
C. 8,24
D. 0,824
Solution: Ta có 24 cm2 = 0,24 dm2 Vậy 8 dm2 24 cm2 = 8,24 dm2.
Correct answer: C. 8,24

Câu hỏi:
10% của 11,5m2 là?
A. 10,15dm2
B. 1,5m2
C. 15,5m2
D. 1,15m2
Solution: 10% của 11,5m2 là: 11,5 ${\\times}$ 10 : 100 = 1,15 (m2).
Correct answer: D. 1,15m2

Câu hỏi:
10% của 11,5m2 là?
A. 10,15dm2
B. 1,5m2
C. 15,5m2
D. 1,15m2
Solution: 10% của 11,5m2 là: 11,5 ${\\times}$ 10 : 100 = 1,15 (m2).   
Correct answer: D. 1,15m2
'''

In [10]:
# prompt = '''
# You are a virtual assistant capable of answering math questions honestly and accurately, without fabricating additional content.
# Based on the following multiple choice questions, choose the correct answer and explain your answer.

# Câu hỏi 1:
# Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính?
# A. 80,000 : 6
# B. 80,000 x 6
# C. 80,000 : (6 x 100)
# D. (80,000 x 6) : 100
# Correct answer: D. (80,000 x 6) : 100
# Explanation: Số tiền lỗ bằng 6% của 80 000 đồng . Số tiền lỗ = 80 000 / 6% = (80,000 x 6) : 100

# Câu hỏi 2:
# 8 dm2 24 cm2 = ……… dm2. Số thích hợp điền vào chỗ chấm là?
# A. 824
# B. 82,4
# C. 8,24
# D. 0,824

# Correct answer: C. 8,24
# Explanation: Ta có 24 cm2 = 0,24 dm2\n Vậy 8 dm2 24 cm2 = 8,24 dm2.


# Câu hỏi 3:
# 10% của 11,5m2 là?
# A. 10,15dm2
# B. 1,5m2
# C. 15,5m2
# D. 1,15m2
# Correct answer: D. 1,15m2
# Explanation: 10% của 11,5m2 là: 11,5 ${\\times}$ 10 : 100 = 1,15 (m2).

# Câu hỏi 4:
# Một tam giác có đáy bằng 15 cm và gấp 2,5 lần chiều cao tương ứng. Diện tích tam giác đó là?
# A. 90 cm2
# B. 37,5 cm2
# C. 45 cm2
# D. 18,75 cm2

# Correct answer: C. 45 cm2
# Explanation: Chiều cao của tam giác đó là: 15 : 2,5 = 6 (cm). Diện tích tam giác đó là: 15 ${\\times}$ 6 : 2 = 45 (cm2). Đáp số: 45 cm2.

# Câu hỏi 5:
# Số 13 076 làm tròn đến chữ số hàng chục nghìn ta được số?
# A. 10 000
# B. 13 000
# C. 13 050
# D. 20 000

# Correct answer: A. 10 000
# Explanation: Số 13 076 gần với số 10 000 hơn số 20 000. Vậy khi làm tròn số 13 076 đến hàng chục nghìn ta được số 10 000.

# Câu hỏi 6:

# '''

In [11]:
file_test = open(PATH_TEST_CSV, 'r')
data_test = json.load(file_test)
file_train = open(PATH_TRAIN_CSV, 'r')
data_train = json.load(file_train)

In [12]:
def generate_prompt_based_on_train(current_question, choices_str, index_search, prefix_prompt,data_train, num_selected = 20):
    #Searching
    sentences = [tokenize(current_question)]
    embedding =  model_embedding.encode(sentences, show_progress_bar = False)
    embedding = np.array(embedding).astype('float32')
    f_dists, f_ids = search_faiss.search(embedding.reshape(1, -1), k=num_selected)
    f_ids = f_ids[0]
    prompt = prefix_prompt
    num_used = 2
    for idx in f_ids:
        #select question in training csv 
        selected_question = data_train[idx]["question"].replace("\n", "").replace("\t", "")
        selected_choices = "\n".join(data_train[idx]["choices"])
        selected_correct_answer = data_train[idx]["answer"]
        if "explanation" in data_train[idx]:
            if num_used == 0:
                break
            selected_explanation = data_train[idx]["explanation"].replace("\n", "").replace("\t", "")
            prompt += "\n\n" + "Câu hỏi: " + selected_question + "\n" +\
                    selected_choices + "\n" + "Solution: " +  selected_explanation\
                    + "\n" + "Correct answer: " + selected_correct_answer
            num_used -= 1
        else:
            continue
#             prompt += "\n\n" + "Câu hỏi: " + selected_question + "\n" +\
#                     selected_choices + "\n" + "Solution: "\
#                     + "\n" + "Correct answer:" + selected_correct_answer
    prompt += "\n\n" + "Câu hỏi: " + current_question + "\n" +\
              choices_str + "\nSolution:"
    return prompt
        

In [13]:
def find_string(text, substring):
    indices = []
    index = text.find(substring)
    while index != -1:
        indices.append(index)
        index = text.find(substring, index + 1)
    return indices



In [14]:
# l_used_idx = [1,3,15,30,78,117,123,125,137,145,151,170]

In [15]:
# print(len(l_used_idx))

In [16]:
l_outputs_fewshot_prompts = []
for idx, item in tqdm(enumerate(data_test["data"])):
    question = item["question"]
    question = question.replace("\n", "").replace("\t", "")
    if question[-1] == ":":
        question = question[:-1] + "?"
    else:
        question = question + "?"
    choices = item["choices"]
    cleaned_choices = []
    for choice in choices: 
        if choice[0] in ['A', 'B', 'C', 'D']:
            cleaned_choices.append(choice)
    choices_str = "\n".join(cleaned_choices)
    prompt = generate_prompt_based_on_train(question,\
                                            choices_str,\
                                            search_faiss,\
                                            prefix_prompt,\
                                            data_train["data"])
#     print(question)
#     print(choices_str)
#     print("PROMPT: ")
#     print(prompt)
    print("**********")
    inputs = tokenizer([prompt], return_tensors="pt").to('cuda')
    res = model.generate(**inputs,  max_new_tokens=200,temperature=0.01)
    output = tokenizer.decode(res.cpu()[0], skip_special_tokens=True)
#     start_idx = find_string(output, "Correct answer:")[-1] + len("Correct answer:")
#     print(output)
    l_outputs_fewshot_prompts.append(output)
    print("##################################################")

0it [00:00, ?it/s]

**********


/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
1it [00:34, 34.46s/it]

##################################################
**********


2it [00:55, 26.59s/it]

...

##################################################
**********


188it [1:06:16, 21.08s/it]

##################################################
**********


189it [1:06:37, 21.15s/it]

##################################################


In [17]:
# def fewshot_basedon_train(data_test):
#     l_outputs_fewshot_prompts = []
#     for item in tqdm(data_test["data"]):
#         question = item["question"]
#         question = question.replace("\n", "").replace("\t", "")
#         if question[-1] == ":":
#             question = question[:-1] + "?"
#         elif question[-1] != "?":
#             question = question[:-1] + "?"
#         choices = item["choices"]
#         cleaned_choices = []
#         for choice in choices: 
#             if choice[0] in ['A', 'B', 'C', 'D']:
#                 cleaned_choices.append(choice)
#         choices_str = "\n".join(cleaned_choices)

#         input_str = prompt + question + "\n" + choices_str + "\n\n" + "Correct answer:"
#         inputs = tokenizer([input_str], return_tensors="pt").to('cuda')
#         res = model.generate(**inputs,  max_new_tokens=50,temperature=0.1)
#         output = tokenizer.decode(res.cpu()[0], skip_special_tokens=True)
        
    
    
    

In [18]:
# l_outputs_fewshot_prompts = []
# for item in tqdm(data_test["data"]):
#     question = item["question"]
#     question = question.replace("\n", "").replace("\t", "")
#     if question[-1] == ":":
#         question = question[:-1] + "?"
#     elif question[-1] != "?":
#         question = question[:-1] + "?"
#     choices = item["choices"]
#     cleaned_choices = []
#     for choice in choices: 
#         if choice[0] in ['A', 'B', 'C', 'D']:
#             cleaned_choices.append(choice)
#     choices_str = "\n".join(cleaned_choices)
#     #print("QUESTION MULTI CHOICES: ", question + "\n" + choices_str)
#     input_str = prompt + question + "\n" + choices_str + "\n\n" + "Correct answer:"
#     inputs = tokenizer([input_str], return_tensors="pt").to('cuda')
#     res = model.generate(**inputs,  max_new_tokens=50,temperature=0.01)
#     output = tokenizer.decode(res.cpu()[0], skip_special_tokens=True)
# #     start_idx = output.index("Câu hỏi 4:") + len("Câu hỏi 4:")
# #     tmp_output = output[start_idx:]
# #     answer_and_explained_idx = tmp_output.index("Correct answer:") + len("Correct answer:")
    
#     responce = output[len(input_str):]
#     if "Câu hỏi" in responce:
#         end_idx = responce.index("Câu hỏi")
#         responce = responce[:end_idx]
#     responce = responce.strip()
#     responce = "\n".join(responce.split("\n"))
#     print(question)
#     print("*********")
#     print("Correct answer: ")
#     print(responce)
#     l_outputs_fewshot_prompts.append(responce)
#     print("##############")
    
    

In [19]:
# print(l_outputs_fewshot_prompts[10])

In [20]:
import json 
json_str = json.dumps(l_outputs_fewshot_prompts)
file_write = open("/kaggle/working/output_fewshot_searching.json", 'w')
file_write.write(json_str)
file_write.close()